# RAG実行ノートブック (v2: クラスベース)

このノートブックは、FAQファイルに対してRAG処理を実行し、
ベクトル検索、リランキング、LLM回答生成を行います。

**前提条件**:
- `.env`ファイルが設定済み（DB接続、OCI認証情報）
- データベーステーブル（source_documents, chunks）が作成済み
- ドキュメントがデータベースに登録済み（12_data_pipeline_v2.ipynb実行済み）
- Object StorageにFAQファイルがアップロード済み

## 1. 必要なライブラリのインポート

In [ ]:
####################################
###  プロジェクトルートをパスに追加  ###
####################################

import sys
import os

# プロジェクトルートをPythonパスに追加
# notebooksディレクトリから実行されている場合、親ディレクトリをパスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"✓ Project root added to Python path: {project_root}")


####################################
########  必要なライブラリ群 ##########
####################################

# 標準ライブラリ
from datetime import datetime
import pandas as pd

# Oracle Database
import oracledb

# 設定読み込み
from config_loader import ConfigLoader

# RAG処理クラス
from src.rag.vector_searcher import VectorSearcher
from src.rag.reranker import JapaneseReranker
from src.rag.answer_generator import AnswerGenerator
from src.rag.ragas_evaluator import RagasEvaluator
from src.rag.excel_handler import ExcelHandler
from src.rag.rag_pipeline import RAGPipeline

print("✓ All imports successful")

## 2. 設定の読み込みとDB接続

In [ ]:
################
#  設定と接続  #
################

# ConfigLoaderを使用して設定を読み込み
config = ConfigLoader()
config.load_env()

# Oracle Database接続
db_params = config.get_db_params()
connection = oracledb.connect(**db_params)

# OCI設定
oci_config = config.get_oci_config()

# GenAI設定
genai_config = config.get_genai_config()
compartment_id = genai_config['compartment_id']
embedding_model = genai_config['embed_model']

# Object Storage設定
os_config = config.get_object_storage_config()
bucket_name = os_config['bucket_name']

# Object Storageクライアントを作成してnamespaceを取得
os_client = config.get_object_storage_client()
namespace = os_client.get_namespace().data

# GenAIクライアント（共通）
genai_client = config.get_genai_client()

print(f"✓ OCIリソースの情報を読み込みました")
print(f"  - Database: {connection}")
print(f"  - Bucket: {bucket_name}")
print(f"  - Namespace: {namespace}")
print(f"  - Compartment ID: {compartment_id[:20]}...")
print(f"  - Embedding Model: {embedding_model}")

## 3. RAGパラメータ設定（必要に応じて編集）

In [ ]:
############################
#  RAGパラメータ設定  #
############################

# ベクトル検索のパラメータ
TOP_K = 10  # ベクトル検索の上位何件を返却するか。Rerank無効時はこの値がコンテキスト件数

# Rerankのパラメータ (RerankはCPU処理性能に依っては長時間要するため注意)
RERANK_ENABLED = True  # True or False
RERANK_TOP_N = 5  # Rerank後に上位何件をコンテキストとして利用するか

# LLM選択パラメータ
# 以下から選択してください：
# 【Cohere】 "cohere.command-a-03-2025", "cohere.command-r-plus-08-2024"
# 【Meta Llama】 "meta.llama-3.3-70b-instruct"
# 【xAI Grok】 "xai.grok-4-fast-non-reasoning", "xai.grok-4-fast-reasoning", "xai.grok-4"
# 【Google Gemini】 "google.gemini-2.5-pro", "google.gemini-2.5-flash", "google.gemini-2.5-flash-lite"
# 【OpenAI GPT-OSS】 "openai.gpt-oss-20b", "openai.gpt-oss-120b"
CHAT_MODEL = "cohere.command-a-03-2025"  # 使用するLLMモデルを指定

# LLMチューニングパラメータ
MAX_TOKENS = 1000  # 生成する最大トークン数
TEMPERATURE = 0.3  # 温度パラメータ（0-1、創造性の制御。0に近いほど決定的、1に近いほどランダム）
TOP_P = 0.75  # Nucleus samplingのパラメータ（0-1）
FREQUENCY_PENALTY = 0.0  # 頻度ペナルティ（Cohere用、0-1。繰り返しを抑制）
TOP_K_SAMPLING = 0  # Top-Kサンプリング（Cohere用、0で無効）

# 回答生成時のプロンプト
ANSWER_PROMPT = """
参考ドキュメントの情報に基づいて、正確に回答してください。ドキュメントに情報がない場合は、その旨を伝えてください。
回答は簡潔に平文で記載してください。
"""

# FAQファイル名（環境変数から取得、デフォルト: faq.xlsx）
FAQ_OBJECT_NAME = config.get_faq_object_name()

print(f"✓ RAGパラメータ設定値")
print(f"  - Vector Search TOP_K: {TOP_K}")
print(f"  - Rerank Enabled: {RERANK_ENABLED}")
print(f"  - Rerank TOP_N: {RERANK_TOP_N}")
print(f"  - Chat Model: {CHAT_MODEL}")
print(f"  - Max Tokens: {MAX_TOKENS}")
print(f"  - Temperature: {TEMPERATURE}")
print(f"  - FAQ File: {FAQ_OBJECT_NAME}")

## 4. RAGパイプライン構築（コンポーネント初期化とオーケストレータ作成）

In [ ]:
############################
#  RAGコンポーネント初期化  # 
############################

# Phase 1: VectorSearcher（ベクトル検索）
searcher = VectorSearcher(
    db_params=db_params,
    embedding_model=embedding_model,
    genai_client=genai_client,
    compartment_id=compartment_id,
    service_endpoint=genai_config['endpoint']
)

# Phase 2: JapaneseReranker（リランキング）
# モデルロードとデバイス検出は自動
print("\nInitializing JapaneseReranker...")
reranker = JapaneseReranker()
print("✓ JapaneseReranker initialized")

# Phase 3: AnswerGenerator（LLM回答生成）
generator = AnswerGenerator(
    genai_client=genai_client,
    compartment_id=compartment_id
)

# Phase 4: RagasEvaluator（RAGAS評価、オプション）
# 基本版では None にして、評価は別セルで実行
evaluator = None

# Phase 5: ExcelHandler（Object Storage I/O）
# FAQ用のバケット名を取得（OCI_FAQ_BUCKET_NAMEまたはOCI_BUCKET_NAME）
faq_bucket_name = config.get_faq_bucket_name()
excel_handler = ExcelHandler(
    oci_config=oci_config,
    bucket_name=faq_bucket_name,
    namespace=namespace
)

print("\n✓ All 5 components initialized")
print(f"  1. VectorSearcher (model: {embedding_model})")
print(f"  2. JapaneseReranker (model: hotchpotch/japanese-reranker-base-v2)")
print(f"  3. AnswerGenerator (compartment: {compartment_id[:20]}...)")
print(f"  4. RagasEvaluator (skipped for now)")
print(f"  5. ExcelHandler (bucket: {faq_bucket_name}, namespace: {namespace})")


############################
#  プログレスコールバック  # 
############################

def progress_callback(message: str):
    """
    処理進捗を表示
    
    Args:
        message: 進捗メッセージ
    """
    print(f"  {message}")


############################
#  RAGパイプラインオーケストレータ  # 
############################

# Phase 6: RAGPipeline（全コンポーネント統合）
pipeline = RAGPipeline(
    searcher=searcher,
    reranker=reranker,
    generator=generator,
    evaluator=evaluator,  # None
    enable_reranking=RERANK_ENABLED,
    top_k=TOP_K,
    rerank_top_n=RERANK_TOP_N,
    progress_callback=progress_callback
)

print("\n✓ RAGPipeline orchestrator created")
print(f"  - Reranking: {'Enabled' if RERANK_ENABLED else 'Disabled'}")
print(f"  - Vector search top_k: {TOP_K}")
print(f"  - Rerank top_n: {RERANK_TOP_N}")
print("  - Error isolation enabled (individual question failures won't stop processing)")
print("  - Progress tracking enabled")

## 5. FAQファイル読み込み

In [ ]:
############################
#  FAQファイルの読み込み  # 
############################

# Object StorageからFAQファイルを読み込み
faq_df = excel_handler.load_faq(FAQ_OBJECT_NAME)

print(f"✓ FAQ file loaded: {len(faq_df)} questions")
print(f"\nFAQ Data Preview:")
print(faq_df[['id', 'question', 'ground_truth', 'filter']].head())

## 6. RAG処理実行（回答生成）

In [ ]:
############
# メイン処理 #
############

print("\n" + "="*60)
print("Starting RAG Processing...")
print("="*60 + "\n")
print(f"Total questions: {len(faq_df)}")
print(f"Model: {CHAT_MODEL}\n")

# すべての質問をパイプライン処理
# process_batch()は各質問を順次処理し、エラーが発生しても継続
batch_result = pipeline.process_batch(
    questions_df=faq_df,
    model=CHAT_MODEL,
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    frequency_penalty=FREQUENCY_PENALTY,
    top_k=TOP_K_SAMPLING,
    answer_prompt=ANSWER_PROMPT
)

# 処理結果のサマリーを表示
print("\n" + "="*60)
print("Processing Complete")
print("="*60)
print(f"\n📊 Summary:")
print(f"  ✓ Success: {batch_result.successful} questions")
print(f"  ✗ Failed:  {batch_result.failed} questions")
print(f"  📁 Total:   {batch_result.total_questions} questions")
print(f"  ⏱️ Time:    {batch_result.elapsed_time:.2f} seconds")
print(f"  📈 Avg:     {batch_result.elapsed_time/batch_result.total_questions:.2f} seconds/question")

# 処理時間の統計を表示
results_df = batch_result.results_df
print(f"\n【処理時間の統計】")
print(f"  ベクトル検索平均: {results_df['vector_search_time'].mean():.2f}秒")
print(f"  Rerank平均: {results_df['rerank_time'].mean():.2f}秒")
print(f"  回答生成平均: {results_df['generation_time'].mean():.2f}秒")
print(f"  合計平均: {results_df['total_time'].mean():.2f}秒")

# 結果のプレビューを表示
print(f"\n📋 Results Preview:")
print(results_df[['id', 'question', 'answer', 'total_time', 'status']].head())

# 失敗した質問があれば警告
if batch_result.failed > 0:
    print(f"\n⚠ Warning: {batch_result.failed} questions failed to process")
    failed_df = results_df[results_df['status'] == 'failed']
    print("\nFailed questions:")
    print(failed_df[['id', 'question', 'error']])


############################
#  メタデータの作成  # 
############################

# メタデータを作成（パフォーマンス統計を含む）
metadata = {
    'パラメータ': [
        'TOP_K (ベクトル検索件数)',
        'RERANK_ENABLED (Rerankが有効か)',
        'RERANK_TOP_N (Rerank後件数)',
        'CHAT_MODEL (使用LLMモデル)',
        'MAX_TOKENS (最大トークン数)',
        'TEMPERATURE (温度)',
        'TOP_P (Nucleus sampling)',
        'FREQUENCY_PENALTY (頻度ペナルティ)',
        'TOP_K_SAMPLING (Top-K sampling)',
        'ANSWER_PROMPT (回答生成時の指示文)',
        'embedding_model',
        'rerank_model',
        '実行日時',
        'FAQ件数',
        '成功件数',
        '失敗件数',
        '全体処理時間（秒）',
        '平均処理時間/件（秒）',
        'ベクトル検索平均時間（秒）',
        'Rerank平均時間（秒）',
        '回答生成平均時間（秒）'
    ],
    '設定値': [
        TOP_K,
        RERANK_ENABLED,
        RERANK_TOP_N,
        CHAT_MODEL,
        MAX_TOKENS,
        TEMPERATURE,
        TOP_P,
        FREQUENCY_PENALTY,
        TOP_K_SAMPLING,
        ANSWER_PROMPT,
        embedding_model,
        'hotchpotch/japanese-reranker-base-v2',
        datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        batch_result.total_questions,
        batch_result.successful,
        batch_result.failed,
        f"{batch_result.elapsed_time:.2f}",
        f"{batch_result.elapsed_time/batch_result.total_questions:.2f}",
        f"{results_df['vector_search_time'].mean():.2f}",
        f"{results_df['rerank_time'].mean():.2f}",
        f"{results_df['generation_time'].mean():.2f}"
    ]
}
metadata_df = pd.DataFrame(metadata)


############################
#  Object Storageへ保存  # 
############################

# ファイル名生成（タイムスタンプ付き）
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_filename = f"rag_result_{timestamp}.xlsx"

# Object Storageにアップロード（メタデータ付き）
print(f"\n📤 Saving results to Object Storage...")
excel_handler.save_results(
    results_df=results_df,
    filename=output_filename,
    metadata_df=metadata_df
)

print(f"\n✓ Results saved successfully")
print(f"  - Bucket: {bucket_name}")
print(f"  - Filename: {output_filename}")
print(f"  - Rows: {len(results_df)}")
print(f"  - Sheets: Results, Settings")

## 8. RAGAS評価（オプション）

このセクションでは、生成された回答の品質をRAGASメトリクスで評価します。

**評価メトリクス**:
- **Answer Correctness**: 回答の正確性（LLM判定 + 埋め込み類似度）
- **Context Recall**: コンテキストが正解情報をどれだけカバーしているか

**注意**: 評価には時間がかかる場合があります（LLM呼び出しが必要なため）。

In [ ]:
############################
#  RAGAS評価の実行  #
############################

print("\n" + "="*60)
print("Starting RAGAS Evaluation...")
print("="*60 + "\n")

# RAGAS評価用のLLMモデル（デフォルトは cohere.command-a-03-2025）
RAGAS_LLM_MODEL = "cohere.command-a-03-2025"

# RagasEvaluatorのインスタンスを作成
evaluator = RagasEvaluator(
    oci_config=oci_config,
    compartment_id=compartment_id,
    service_endpoint=genai_config['endpoint'],
    embedding_model=embedding_model,
    llm_model=RAGAS_LLM_MODEL,
    genai_client=genai_client,
    batch_size=3,  # バッチサイズ（小さくするとタイムアウトを回避しやすい）
    max_retries=3,
    retry_wait=30
)

print(f"✓ RagasEvaluator initialized")
print(f"  - LLM Model: {RAGAS_LLM_MODEL}")
print(f"  - Embedding Model: {embedding_model}")
print(f"  - Batch Size: 3")

# results_dfから評価に必要なデータを準備
# 失敗した質問は除外
eval_df = results_df[results_df['status'] == 'success'].copy()

if len(eval_df) == 0:
    print("\n⚠ Warning: No successful results to evaluate")
else:
    # 評価データの準備
    questions = eval_df['question'].tolist()
    answers = eval_df['answer'].tolist()
    ground_truths = eval_df['ground_truth'].tolist()
    
    # contextsの準備（contexts列から評価用のリストに変換）
    # contextsが文字列の場合はリストに変換
    contexts = []
    for chunks in eval_df['contexts'].tolist():
        if isinstance(chunks, list):
            contexts.append(chunks)
        elif isinstance(chunks, str):
            # 文字列の場合はリストに変換
            contexts.append([chunks])
        else:
            # 空の場合はプレースホルダー
            contexts.append([""])
    
    print(f"\nEvaluating {len(eval_df)} questions...")
    print(f"  Questions: {len(questions)}")
    print(f"  Answers: {len(answers)}")
    print(f"  Contexts: {len(contexts)}")
    print(f"  Ground Truths: {len(ground_truths)}")
    
    # RAGAS評価を実行
    try:
        eval_result = evaluator.evaluate(
            questions=questions,
            answers=answers,
            contexts=contexts,
            ground_truths=ground_truths
        )
        
        # 評価結果をDataFrameに追加
        eval_df['answer_correctness'] = eval_result.answer_correctness
        eval_df['context_recall'] = eval_result.context_recall
        
        # 結果のサマリーを表示
        print("\n" + "="*60)
        print("RAGAS Evaluation Complete")
        print("="*60)
        print(f"\n📊 Evaluation Metrics Summary:")
        print(f"  Answer Correctness (平均): {eval_df['answer_correctness'].mean():.4f}")
        print(f"  Answer Correctness (最小): {eval_df['answer_correctness'].min():.4f}")
        print(f"  Answer Correctness (最大): {eval_df['answer_correctness'].max():.4f}")
        print(f"  Context Recall (平均): {eval_df['context_recall'].mean():.4f}")
        print(f"  Context Recall (最小): {eval_df['context_recall'].min():.4f}")
        print(f"  Context Recall (最大): {eval_df['context_recall'].max():.4f}")
        
        # 評価結果のプレビュー
        print(f"\n📋 Evaluation Results Preview:")
        print(eval_df[['id', 'question', 'answer_correctness', 'context_recall']].head())
        
        # 元のresults_dfに評価結果をマージ（失敗した質問は評価スコアがNaNになる）
        results_df = results_df.merge(
            eval_df[['id', 'answer_correctness', 'context_recall']],
            on='id',
            how='left'
        )
        
        ############################
        #  Object Storageへ再保存  #
        ############################
        
        # ファイル名生成（評価結果付き）
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_filename_with_eval = f"rag_result_with_ragas_{timestamp}.xlsx"
        
        # Object Storageにアップロード（評価結果付き）
        # メタデータは回答生成セルで作成したものをそのまま使用
        print(f"\n📤 Saving results with RAGAS evaluation to Object Storage...")
        excel_handler.save_results(
            results_df=results_df,
            filename=output_filename_with_eval,
            metadata_df=metadata_df
        )
        
        print(f"\n✓ Results with RAGAS evaluation saved successfully")
        print(f"  - Bucket: {bucket_name}")
        print(f"  - Filename: {output_filename_with_eval}")
        print(f"  - Rows: {len(results_df)}")
        print(f"  - Sheets: Results, Settings")
        print(f"  - New Columns: answer_correctness, context_recall")
        
    except Exception as e:
        print(f"\n❌ RAGAS evaluation failed: {str(e)}")
        print("  You can continue without evaluation results.")
        import traceback
        traceback.print_exc()

## 9. DB接続のクローズ

In [ ]:
################
#  後処理  # 
################

# データベース接続を明示的にクローズ
connection.close()
print("✓ Database connection closed")